# Secret in AWS Parameter Store

AWS Parameter Store is a Free service allows you to securely store parameters. This is how you deploy parameter:

In [1]:
import boto3
from pysecret import Parameter, deploy_parameter, delete_parameter
from rich import print as rprint

ssm_client = boto3.client("ssm")

param_name = "pysecret-demo"

_ = delete_parameter(ssm_client, param_name) # ensure it is not exists at beginning

## Create a new parameter

In [2]:
param = deploy_parameter(
    ssm_client,
    name=param_name,
    data={"password": "mypassword1"},
    type_is_secure_string=True,
    use_default_kms_key=True,
    tier_is_intelligent=True,
    tags=dict(EnvName="dev"),
)
# it returns a in-memory presentation of the deployed parameter
rprint(param)

Parameter(
    Name='pysecret-demo',
    Value='{"password": "mypassword1"}',
    Type='SecureString',
    Version=1,
    LastModifiedDate=datetime.datetime(2023, 11, 18, 15, 2, 51),
    DataType='text',
    ARN=None,
    Selector=None,
    SourceResult=None,
    Tags={},
    Labels=[]
)

## Update a parameter

When you update parameter, it creates a new version of this parameter, [AWS only keep the last 100 versions](https://docs.aws.amazon.com/systems-manager/latest/userguide/sysman-paramstore-versions.html).

In [3]:
param = deploy_parameter(
    ssm_client,
    name=param_name,
    data={"password": "mypassword2"},
    type_is_secure_string=True,
    use_default_kms_key=True,
    tier_is_intelligent=True,
    tags=dict(EnvName="prod"), # also update tag
    overwrite=True, # set overwrite = True to allow update
    skip_if_duplicated=True, # if True, then it won't update if the parameter data is the same
)
# it returns a in-memory presentation of the updated parameter
rprint(param)

Parameter(
    Name='pysecret-demo',
    Value='{"password": "mypassword2"}',
    Type='SecureString',
    Version=2,
    LastModifiedDate=datetime.datetime(2023, 11, 18, 15, 2, 51),
    DataType='text',
    ARN=None,
    Selector=None,
    SourceResult=None,
    Tags={},
    Labels=[]
)

## Don't Update if the Content Doesn't Change

AWS will create a new version regardless if your parameter data is changed. To avoid this, ``pysecret`` will not deploy if the parameter data is not changed.

In [4]:
param = deploy_parameter(
    ssm_client,
    name=param_name,
    data={"password": "mypassword2"},
    type_is_secure_string=True,
    use_default_kms_key=True,
    tier_is_intelligent=True,
    tags=dict(EnvName="prod"), # also update tag
    overwrite=True, # set overwrite = True to allow update
    skip_if_duplicated=True, # if True, then it won't update if the parameter data is the same
)
rprint(param)

None

## Read parameter data

In [5]:
param = Parameter.load(
    ssm_client, 
    name=param_name, 
    with_decryption=True,
)
rprint(param)

Parameter(
    Name='pysecret-demo',
    Value='{"password": "mypassword2"}',
    Type='SecureString',
    Version=2,
    LastModifiedDate=datetime.datetime(2023, 11, 18, 10, 2, 51, 335000, tzinfo=tzlocal()),
    DataType='text',
    ARN='arn:aws:ssm:us-east-1:111122223333:parameter/pysecret-demo',
    Selector=None,
    SourceResult=None,
    Tags={},
    Labels=[]
)

In [6]:
# access the AWS account id of the parameter
param.aws_account_id

'111122223333'

In [7]:
# access the AWS region of the parameter
param.aws_region

'us-east-1'

In [8]:
# access the raw data
param.Value

'{"password": "mypassword2"}'

In [9]:
# access the parameter data in different form
param.json_dict

{'password': 'mypassword2'}

In [10]:
# access the parameter data in JSON format
# there are more format options
# - string
# - string_list
# - json_dict
# - json_list
# - py_object, when the parameter is a pickled object 
rprint(param.json_dict)

{'password': 'mypassword2'}

## Store Arbitrary Python Object

``pysecret`` can even store arbitrary Python object in AWS Parameter Store, it uses the [jsonpickle](https://pypi.org/project/jsonpickle/) library to serialize Python object into string. You have to install ``jsonpickle`` to use this feature.

In [11]:
import dataclasses

@dataclasses.dataclass
class Env:
    username: str = dataclasses.field()
    password: str = dataclasses.field()


@dataclasses.dataclass
class Config:
    dev: Env = dataclasses.field()
    prod: Env = dataclasses.field()

In [12]:
config = Config(
    dev=Env(
        username="dev-username",
        password="dev-password",
    ),
    prod=Env(
        username="prod-username",
        password="prod-password",
    ),
)

param_name = "pysecret-pyobject"

deploy_parameter(
    ssm_client,
    name=param_name,
    data=config,
    type_is_string=True,
    tier_is_intelligent=True,
    overwrite=True,
)

param = Parameter.load(ssm_client, name=param_name)
rprint(param.py_object) # access the underlying data

Config(
    dev=Env(username='dev-username', password='dev-password'),
    prod=Env(username='prod-username', password='prod-password')
)

## Delete Parameter

You can delete the parameters to keep your AWS Account clean

In [13]:
delete_parameter(ssm_client, "pysecret-demo");
delete_parameter(ssm_client, "pysecret-pyobject");

## Parameter Version

In [14]:
param_name_version_example = "pysecret-version"
delete_parameter(ssm_client, param_name_version_example)
for password in ["pass1", "pass2", "pass3"]:
    param = deploy_parameter(
        ssm_client,
        name=param_name_version_example,
        data={"password": password},
        type_is_secure_string=True,
        use_default_kms_key=True,
        tier_is_intelligent=True,
        tags=dict(EnvName="prod"), # also update tag
        overwrite=True, # set overwrite = True to allow update
        skip_if_duplicated=True, # if True, then it won't update if the parameter data is the same
    )
    rprint(param.Version)

1

2

3

In [15]:
# get the version 1
param = Parameter.load(
    ssm_client, 
    name=param_name_version_example, 
    version=1,
    with_decryption=True,
)
rprint(param)

Parameter(
    Name='pysecret-version',
    Value='{"password": "pass1"}',
    Type='SecureString',
    Version=1,
    LastModifiedDate=datetime.datetime(2023, 11, 18, 10, 2, 52, 407000, tzinfo=tzlocal()),
    DataType='text',
    ARN='arn:aws:ssm:us-east-1:111122223333:parameter/pysecret-version',
    Selector=':1',
    SourceResult=None,
    Tags={},
    Labels=[]
)

In [16]:
# get the latest version
param = Parameter.load(
    ssm_client, 
    name=param_name_version_example, 
    # to get the latest version, don't specify the version
    with_decryption=True,
)
rprint(param)

Parameter(
    Name='pysecret-version',
    Value='{"password": "pass3"}',
    Type='SecureString',
    Version=3,
    LastModifiedDate=datetime.datetime(2023, 11, 18, 10, 2, 52, 867000, tzinfo=tzlocal()),
    DataType='text',
    ARN='arn:aws:ssm:us-east-1:111122223333:parameter/pysecret-version',
    Selector=None,
    SourceResult=None,
    Tags={},
    Labels=[]
)

## Parameter Label

[A parameter label is a user-defined alias to help you manage different versions of a parameter.](https://docs.aws.amazon.com/systems-manager/latest/userguide/sysman-paramstore-labels.html).

In [17]:
param_name_label_example = "pysecret-label"
for password, label in [
    ("pass1", "dev"), 
    ("pass2", "test"), 
    # You can't attach the same label to different versions of the same parameter.
    # if the label is already attached to a version, 
    # it will automatically detach, then attach to the new version
    ("pass3", "prod"),
    ("pass4", "prod"),
    ("pass5", "prod"),
]:
    param = deploy_parameter(
        ssm_client,
        name=param_name_label_example,
        data={"password": password},
        type_is_secure_string=True,
        use_default_kms_key=True,
        tier_is_intelligent=True,
        overwrite=True, # set overwrite = True to allow update
        skip_if_duplicated=True, # if True, then it won't update if the parameter data is the same
    )
    if param is not None:
        param.put_label(ssm_client, labels=[label])

In [18]:
param = param.load(
    ssm_client, 
    name=param_name_label_example, 
    label="dev",
)
rprint(param)

Parameter(
    Name='pysecret-label',
    Value='{"password": "pass1"}',
    Type='SecureString',
    Version=1,
    LastModifiedDate=datetime.datetime(2023, 11, 18, 10, 2, 53, 253000, tzinfo=tzlocal()),
    DataType='text',
    ARN='arn:aws:ssm:us-east-1:111122223333:parameter/pysecret-label',
    Selector=':dev',
    SourceResult=None,
    Tags={},
    Labels=['dev']
)

In [19]:
param = param.load(
    ssm_client, 
    name=param_name_label_example, 
    label="prod",
)
rprint(param)

Parameter(
    Name='pysecret-label',
    Value='{"password": "pass5"}',
    Type='SecureString',
    Version=5,
    LastModifiedDate=datetime.datetime(2023, 11, 18, 10, 2, 54, 180000, tzinfo=tzlocal()),
    DataType='text',
    ARN='arn:aws:ssm:us-east-1:111122223333:parameter/pysecret-label',
    Selector=':prod',
    SourceResult=None,
    Tags={},
    Labels=['prod']
)

## Clearn Up

Now you know how to use ``pysecret`` to manage AWS Parameter. Let's clean up the AWS resources created in this tutorial.

In [21]:
_ = delete_parameter(ssm_client, param_name_version_example)
_ = delete_parameter(ssm_client, param_name_label_example)